In [3]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from fractions import Fraction
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast

In [4]:
# parameters
customer_id = '90'
formatted_attribute = 'width'
customer_name='%wbmason%'
buckets = """Office Chairs"""

strategy_version_input=747
attribute_id_input=2426

In [5]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [6]:
bucketz = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [7]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text
# _______________________________________________________________________________________________

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

# def find_first_digit(s, non=False):
#     for i, x in enumerate(s):
#         if x.isdigit() ^ non:
#             return i
#     return -1

# def split_digits(s, case=False):
#     non = True
#     while s:
#         i = find_first_digit(s, non)
#         if i == 0:
#             non = not non
#         elif i == -1:
#             yield int(s) if s.isdigit() else s if case else s.lower()
#             s = ''
#         else:
#             x, s = s[:i], s[i:]
#             yield int(x) if x.isdigit() else x if case else x.lower()

# def natural_key(s, *args, **kwargs):
#     return tuple(split_digits(s, *args, **kwargs))

# _______________________________________________________________________________________________

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
    
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
        
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))


def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))

# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

In [8]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df = df[df['bucket_id'].isin(bucketz)]
len(df)

12300

In [ ]:
regex_pattern=r'(?i)(\d+.?\d+\/\d+["]\W?W(?!ith)(?!eigh)(?!ooden)i?d?t?h?\b)|(\d+\.\d*\"\W?W(?!ith)(?!eigh)(?!ooden)i?d?t?h?\b)|(\d+\"\s?W(?!ith)(?!eigh)(?!ooden)i?d?t?h?\b)|((?<!\/)(?<!\.)\d+.?["]\W?W(?!ith)(?!eigh)(?!ooden)i?d?t?h?\b)|(\d+\/\d+["]\W?W(?!ith)(?!eigh)(?!ooden)i?d?t?h?\b)|()'
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

regex_pattern_blank=r'\d'
df['new_matches'] = df['matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))



df_width=df[(df['matches'].astype(str)!='[]')&(df['new_matches'].astype(str)!='[]')]
df_width['new_matches']=df_width['matches'].apply(lambda x: re.sub(r'(?i)\\" W|"W|" W',' in',str(x))).apply(lambda x: re.sub(r'(?i)\\" W|"W|\"xa0W|xa0W|"|idth','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
df_width['new_matches']=df_width['new_matches'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])

df_width['new_matches']=df_width['new_matches'].apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub(",",'","',str(x))).apply(lambda x: re.sub('" ','"',str(x))).apply(lambda x: re.sub('","',' in","',str(x))).apply(lambda x: re.sub('"]',' in"]',str(x))).apply(lambda x: re.sub('in in','in',str(x)))                                                                          
df_width['new_matches']=df_width['new_matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
df_width['new_matches']=df_width['new_matches'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
df_width['new_matches']=df_width['new_matches'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))

df_width[curation_col]=df_width['new_matches']
matches_natural=df_width[['external_id',curation_col]]
print(len(matches_natural))
# matches_natural

In [ ]:
df_width[df_width['external_id'].astype(str)=='12.0 in']

In [ ]:
df_width[curation_col]=df_width[curation_col].apply(lambda x: re.sub(r'''12.0 in''','12 in',str(x)))
df_width[curation_col]=df_width[curation_col].apply(lambda x: re.sub(r'''125''','13',str(x)))
df_width[curation_col]=df_width[curation_col].apply(lambda x: re.sub(r'''3.12 in","3.13 in''','3.12 in',str(x)))
df_width[curation_col]=df_width[curation_col].apply(lambda x: re.sub(r'''4.12 in","4.13 in''','4.12 in',str(x)))
df_width[curation_col]=df_width[curation_col].apply(lambda x: re.sub(r'''875''','88',str(x)))
df_width[curation_col]=df_width[curation_col].apply(lambda x: re.sub(r'''625''','63 in',str(x)))

 
matches_width=df_width[['external_id',curation_col]]
# matches_width[curation_col]=matches_width[curation_col].apply(lambda x:re.sub(r'''in''',' in',str(x)))
matches_width

# Find more values

In [ ]:
pat=r'(\d+.?\d?\/?\d*\\\"\s?W(?!eigh)\b)|(\d+\.\d*\\\"\s?W(?!eigh)\b)|()'
df['matches'] = df['long_desc'].apply(lambda x: re_extract(pat, x))
regex_pattern_length=r'(?i)(\d+\.\d+.?.?.?Le?n?g?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Le?n?g?t?h?\b)|()'         
df['matches_length'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_height=r'(?i)(\d+\.\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b)|((?<!\.)(?<!up.to.)(?<![\w])(?<!\/)(?<!\/)\d+(?!\.\d)(?<!\d\\?\"?\-\d\d)[\W]?[\W]?[\W]?He?i?g?h?t?(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'        
df['matches_height'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
regex_pattern_width=r'(?i)(\d+\.\d+.?.?.?Wi?d?t?h?(?!eigh)\b)|(\d+.?\d+\/\d+.?.?.?Wi?d?t?h?(?!eigh)\b)|(\b(?<!\.)(?<!\/)\d+(?!\.\d)[\W]?[\W]?Wi?d?t?h?(?!eigh)(?!\/)\b)|(\b(?<!\.)(?<!\/)\d+(?!\.)[\W]?[\W]?Wi?d?t?h?(?!eigh)(?!\/)\b)|()'
df['matches_width'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
regex_pattern_diameter=r'(?i)(\d+\.\d+.?.?.?Diam?e?t?e?r?\b)|(\d+.?\d+\/\d+.?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Diam?e?t?e?r?\b)|()'
df['matches_diameter'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
regex_pattern_ft=r"\d'.?.?.?[LWHD]|()"
df['matches_ft'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
regex_pattern_blank=r'\d|()'
df['new_matches'] = df['matches_length'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'(?i)(\d+.?\d*\/?\d*.?I?n?c?h?"?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?)|(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df['blank_matches'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df['blank_matches_long'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

In [ ]:
df_more=df[(df['matches'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')&(df['matches_width'].astype(str)!='[]')]
print(len(df_more))
# df_more

In [ ]:
df_more['matches_more']=df_more['matches_width'].apply(lambda x: re.sub(r'''\\" W|".?W|''.?W|”.?W|˝.?W|”.?W|” w|″ W''',' in',str(x))).apply(lambda x: re.sub(r'''\\' W|'.?W|'.?W''',' ft',str(x))).apply(lambda x: re.sub(r'\\" W|"W|\"xa0W|xa0W|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))

df_more['matches_more']=df_more['matches_more'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])

In [ ]:
df_more['matches_more']=df_more['matches_more'].apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub(",",'","',str(x))).apply(lambda x: re.sub('" ','"',str(x))).apply(lambda x: re.sub('","',' in","',str(x))).apply(lambda x: re.sub('"]',' in"]',str(x))).apply(lambda x: re.sub('in in','in',str(x))).apply(lambda x: re.sub('ft in','ft',str(x)))                                                                           
df_more['matches_more']=df_more['matches_more'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
df_more['matches_more']=df_more['matches_more'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
df_more['matches_more']=df_more['matches_more'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))

df_more[curation_col]=df_more['matches_more']
matchesmore=df_more[['external_id',curation_col]]
print(len(matchesmore))

In [ ]:
df_na=df[(df['matches'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')&(df['matches_width'].astype(str)=='[]')]
df_na[curation_col]='n/a'
matches_na=df_na[['external_id',curation_col]]
# matches_na

# NA

In [ ]:
df_na=df[(df['matches_length'].astype(str)=='[]')&(df['matches_height'].astype(str)=='[]')&(df['matches_width'].astype(str)=='[]')&(df['matches_ft'].astype(str)=='[]')&(df['matches_diameter'].astype(str)=='[]')&(df['blank_matches'].astype(str)=='[]')&(df['blank_matches_long'].astype(str)=='[]')]#&(df['new_matches'].astype(str)!='[]')]
df_na[curation_col]='n/a'
print(len(df_na))
# df_na

In [ ]:
matches_df=df_na[['external_id',curation_col]]
print(len(matches_df))
matches_df

In [ ]:
# # Two
# (?i)(?<!x)\W\W\d+.?\d*\/?\d*.?I?n?c?h?"?'?\W?[lwdh]?.?x.?\d+.?\d*\/?\d*\WI?n?c?h?"?'?\W?[lwdh]?\W?\W?\W?\W(?!x)
#  #Three
# |(\d+.?\d*\/?\d*.?I?n?c?h?"?'?\W?[lwdh]?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?'?\W?[lwdh]?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?'?\W?[lwdh]?)
# # All
# (\d+.?\d*\/?\d*.?I?n?c?h?"?'?\W?[lwdh]?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?'?\W?[lwdh]?.?x?.?\d*.?\d*\/?\d*.?I?n?c?h?"?'?\W?[lwdh]?)

In [ ]:
# all_split=r'''(?i)(\d+\W?\d*\/?\d*\W?I?n?c?h?"?'?f?t?\W?[lwdh]?\W?x\W?\d+\W?\W?\d*\/?\W?\d*\W?I?n?c?h?"?'?f?t?\W?[lwdh]?\W?\W?x?\W?\d*\W?\d*\/?\d*\W?I?n?c?h?"?'?f?t?\W?([lwdh]\b)?)|()'''
# pat_split_THREE=r'''(?i)(\W?\W?\d+.?\d*\/?\d*.?I?n?c?h?"?'?f?t?\W?[lwdh]?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?'?f?t?\W?[lwdh]?.?x.?\d*.?\d+\/?\d*.?I?n?c?h?"?'?f?t?\W?[lwdh]?\w?)|()'''
# pat_split_TWO=r'''(?i)(\[\"\d+\.?\d*.{0,4}x\"\d+\.?\d*.{0,4}"\])|()'''

# Min Max (Min is width)

In [9]:
all_split=r'''(?i)(\d+\W?\d*\/?\d*\W?I?n?c?h?"?'?f?t?\W?[lwdh]?\W?x\W?\d+\W?\W?\d*\/?\W?\d*\W?I?n?c?h?"?'?f?t?\W?[lwdh]?\W?\W?x?\W?\d*\W?\d*\/?\d*\W?I?n?c?h?"?'?f?t?\W?([lwdh]\b)?)|()'''
pat_split_THREE=r'''(?i)(\W?\W?\d+.?\d*\/?\d*.?I?n?c?h?"?'?f?t?\W?[lwdh]?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?'?f?t?\W?[lwdh]?.?x.?\d*.?\d+\/?\d*.?I?n?c?h?"?'?f?t?\W?[lwdh]?\w?)|()'''
pat_split_TWO=r'''(?i)(\[\"\d+\.?\d*.{0,4}x\"\d+\.?\d*.{0,4}"\])|()'''

df['matches_split']=df['product_name'].apply(lambda x: re_extract(all_split, str(x)))
df['matches_split_frac']=df['matches_split'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))])
df['matches_split_frac']=df['matches_split_frac'].apply(lambda x: re.sub(r'''�''','',str(x))).apply(lambda x: re.sub(r'''(\[\"\[\')|(\[\'\[\")|(\[\"\[\\\')''',"['[\'",str(x))).apply(lambda x: re.sub(r'''(\'\]\"\])|(\\\'\]\"\])|(\"\]\'\])''','''\']']''',str(x))).apply(lambda x: re.sub(r'''\"''',' in',str(x))).apply(lambda x: re.sub(r'''(\[\'\[\\\')|(\[\'\[\')''','["',str(x))).apply(lambda x: re.sub(r'''(\\\'\]\'\])|(\'\]\'\])''','"]',str(x))).apply(lambda x: re.sub(r'''(?i)x''','"x"',str(x))).apply(lambda x: re.sub(r'''\'''',' ft',str(x))).apply(lambda x: re.sub(r''' ''','',str(x))).apply(lambda x: re.sub(r'''\\ft\\ft''','ft',str(x))).apply(lambda x: re.sub(r'''\\ft''','ft',str(x))).apply(lambda x: re.sub(r'''X''','x',str(x))).apply(lambda x: re.sub(r'''in''',' in',str(x))).apply(lambda x: re.sub(r'''ft''',' ft',str(x))).apply(lambda x: re.sub(r'''\,.*''','"]',str(x)))                                                                                                                                                             

df['matches_split_three'] = df['matches_split_frac'].apply(lambda x: re_extract(pat_split_THREE, str(x)))
df['matches_split_two'] = df['matches_split_frac'].apply(lambda x: re_extract(pat_split_TWO, str(x)))


regex_pattern_blank=r'(?i)(\d+\.\d*.?w\b)|()'
df['matches'] = df['matches_split_frac'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
split=df[df['matches_split'].astype(str)!='[]']
print(len(split))
print(len(split[split['matches_split_two'].astype(str)!='[]']))
print(len(split[split['matches_split_three'].astype(str)!='[]']))

7445
4985
1829


In [12]:
pat_call_cout=r'''(?i)(\d+\.?\d*.?.?w\b)|()'''
split['matches']=split['matches_split_frac'].apply(lambda x: re_extract(pat_call_cout, str(x)))

print(len(split[split['matches'].astype(str)!='[]']))
width_call_out=split[split['matches'].astype(str)!='[]']
width_call_out['matches']=width_call_out['matches'].apply(lambda x: re.sub(r'''(?i)w''',' in',str(x))).apply(lambda x: re.sub(r'''\[''','["',str(x))).apply(lambda x: re.sub(r'''\]''','"]',str(x))).apply(lambda x: re.sub(r'''(')''','',str(x)))
width_call_out[curation_col]=width_call_out['matches']
matcheswidthcallout=width_call_out[['external_id',curation_col]]

332


In [ ]:
# del split['new_matches']
# del split['matches_length']
# del split['matches_height']
# del split['matches_width']
# del split['matches_diameter']
# del split['matches_ft']
# del split['blank_matches']
# del split['blank_matches_long']

In [14]:
# split['matches_split_two'].explode().value_counts()[0:500]
print(len(split[split['matches_split_two'].astype(str)!='[]']))
split_two=split[(split['matches_split_two'].astype(str)!='[]')&(split['matches'].astype(str)=='[]')]
print(len(split_two))

4985
4950


In [15]:
split_two

product_id external_id  \
4       434387647  5637232194   
6       434380627  5637178305   
8       434380616  5637178293   
12      434473365  5637439234   
13      434387396  5637218289   
...           ...         ...   
12410   434383340  5637209919   
12412   510951928  5637488979   
12417   434481628  5637147418   
12424   434382115  5637181774   
12426   434492186  5637156997   

                                                                                   product_name  \
4         Heritage Accufit Can Liners, Prime Resin, 23 gal, 0.9 mil, 28 x 45, Black, 200/Carton   
6      Earthsense Commercial Recycled Can Liners, 31-33gal, 1.65mil, 33 x 39, Black, 100/Carton   
8             Platinum Plus Can Liner, Super Hexene Resin 31-33gal, 1.35mil, 33 x 40, 50/Carton   
12                                        W.B. Mason Co. Metal Platform Truck, 24" x 48", Green   
13                                  RePrime Can Liners, 23gal, .90mil, Clear, 28" x 45", 50/Box   
...                                                                                         ...   
12410                     RePrime Can Liners, Prime Resin, 37 x 50, 1.3 mils, Black, 100/Carton   
12412                                             Brother CZ Roll Cassette, 1" x 16.4 ft, White   
12417       Avery Large Tent Cards, Uncoated, Embossed, Two-Sided Printing, 3 1/2" x 11", 50/BX   
12424                Flagship Premium Copy Paper, 92 Bright, 20 lb., 8 1/2 x 11, White, 5000/CT   
12426         HP Papers Multipurpose 20lb Copy Paper, 8.5" x 11", 96 Bright, 1 Ream, 500 Sheets   

                                                                                                                                                                                                                                                                                                                                                                                                                                        long_desc  \
4                                                                                                                                                                                                                         Accufit Can Liners, Prime Resin, 23 gal, 0.9 mil, 28 x 45, Black, 200/Carton•Constructed of prime-quality resin.•Accufit sizing ensures that bags won't fall into can.•Designed for most common sizes and applications.   
6                                                                                                                                                                Recycled Can Liners, 31-33gal, 1.65mil, 33 x 39, Black, 100/Carton•Superior puncture resistance•Extra heavy-duty liners meet EPA CPG for recycled content•Convenient dispenser carton•Certified to qualify for LEED credits•Star-seal bottom for leak resistance; coreless rolls   
8                                                                                                                                                               Can Liner, Super Hexene Resin 31-33gal, 1.35mil, 33 x 40, 50/Carton•Strong resin blend is designed for toughest use.•Superior puncture resistant sidewall strength.•Star bottom design distributes load evenly and helps prevent leaks.•Coreless rolls for convenient dispensing.   
12                                                                                                                                                                      Metal Platform Truck, 24" x 48", Green•Constructed from sturdy 12 gauge steel to withstand years of use•Removable cross-brace handle can be inserted on either end of the platform•Extra large, 8" diameter wheels, sold separately (BPT-WS1025), are easily maneuverable   
13                                                                                                                                                                                                                       Can Liners, 23gal, 

In [16]:
pat_split_width_inch=r'''(?i)(ft)|()'''
split_two['ft']=split_two['matches_split_two'].apply(lambda x: re_extract(pat_split_width_inch, str(x)))
split_two_inch=split_two[split_two['ft'].astype(str)=='[]']
print(len(split_two_inch))
split_two_inch.head(2)

4385


product_id external_id  \
4   434387647  5637232194   
6   434380627  5637178305   

                                                                               product_name  \
4     Heritage Accufit Can Liners, Prime Resin, 23 gal, 0.9 mil, 28 x 45, Black, 200/Carton   
6  Earthsense Commercial Recycled Can Liners, 31-33gal, 1.65mil, 33 x 39, Black, 100/Carton   

                                                                                                                                                                                                                                                                          long_desc  \
4                                                           Accufit Can Liners, Prime Resin, 23 gal, 0.9 mil, 28 x 45, Black, 200/Carton•Constructed of prime-quality resin.•Accufit sizing ensures that bags won't fall into can.•Designed for most common sizes and applications.   
6  Recycled Can Liners, 31-33gal, 1.65mil, 33 x 39, Black, 100/Carton•Superior puncture resistance•Extra heavy-duty liners meet EPA CPG for recycled content•Convenient dispenser carton•Certified to qualify for LEED credits•Star-seal bottom for leak resistance; coreless rolls   

   customer_id attributes              buckets  bucket_id values  \
4           90       None  Waste / Refuse Bags       6689   None   
6           90       None  Waste / Refuse Bags       6689   None   

  matches_split matches_split_frac matches_split_three matches_split_two  \
4   [28 x 45, ]        ["28"x"45"]                  []     [["28"x"45"]]   
6   [33 x 39, ]        ["33"x"39"]                  []     [["33"x"39"]]   

  matches  ft  
4      []  []  
6      []  []

In [26]:
pat_split_length=r'''(?i)(x\"\d+\.?\d*.{0,4}"\])|()'''
pat_split_width=r'''(?i)(\[\"\d+\.?\d*.{0,4}x)|()'''
split_two_inch['length']=split_two_inch['matches_split_two'].apply(lambda x: re_extract(pat_split_length, str(x)))
split_two_inch['width']=split_two_inch['matches_split_two'].apply(lambda x: re_extract(pat_split_width, str(x)))

In [27]:
split_two_inch['length']=split_two_inch['length'].apply(lambda x: re.sub(r'''\[|\]|x|\"|in|'| ''','',str(x)))
split_two_inch['width']=split_two_inch['width'].apply(lambda x: re.sub(r'''\[|\]|x|\"|in|'| ''','',str(x)))
# split_two_inch

In [63]:
split_two_inch['length']=split_two_inch['length'].apply(lambda x: re.sub(r'''12.5di''','12.5',str(x))).apply(lambda x: re.sub(r'''10\.5\?''','10.5',str(x))).apply(lambda x: re.sub(r'''0\.62\-''','0.62',str(x))).apply(lambda x: re.sub(r'''18\-''','18',str(x)))
split_two_inch['length']=split_two_inch['length'].apply(lambda x: re.sub(r'\?|\-','',str(x)))
split_two_inch['width']=split_two_inch['width'].apply(lambda x: re.sub(r'\?|\-','',str(x)))

In [38]:
pat_call_cout=r'''(?i)(\d+\.?\d*.?.?[a-z]\b)|()'''
split_two_inch['matches']=split_two_inch['length'].apply(lambda x: re_extract(pat_call_cout, str(x)))
split_two_inch=split_two_inch[split_two_inch['matches'].astype(str)=='[]']

In [79]:
# split_two_inch[split_two_inch['length'].astype(str)=='8?']

In [64]:
split_two_inch['length']=split_two_inch['length'].astype(float)
split_two_inch['width']=split_two_inch['width'].astype(float)

split_two_inch['Max'] = split_two_inch[['length', 'width']].values.max(1)
split_two_inch['Min'] = split_two_inch[['length', 'width']].values.min(1)
split_two_inch['Max']=split_two_inch['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

In [68]:
split_two_inch=split_two_inch[split_two_inch['length'].astype(str)!=""]
split_two_inch=split_two_inch[split_two_inch['width'].astype(str)!=""]
split_two_inch['length']=split_two_inch['length'].apply(lambda x:re.sub(r' in','',str(x)))#.apply(lambda x:str(f'"{x}"'))
split_two_inch['width']=split_two_inch['width'].apply(lambda x:re.sub(r' in','',str(x))).apply(lambda x: re.sub(r'','',str(x)))
split_two_inch['length']=split_two_inch['length'].astype(float)
split_two_inch['width']=split_two_inch['width'].astype(float)

split_two_inch['Max'] = split_two_inch[['length', 'width']].values.max(1)
split_two_inch['Min'] = split_two_inch[['length', 'width']].values.min(1)
split_two_inch['Max']=split_two_inch['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
split_two_inch['Min']=split_two_inch['Min'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

In [78]:
# split_two_inch[500:1000]

In [76]:
split_two_inch[curation_col]=split_two_inch['Min']
msplit=split_two_inch[['external_id',curation_col]]
msplit[curation_col]=split_two_inch[curation_col].apply(lambda x: str(f"['{x} in']")).apply(lambda x: re.sub(r"'",'"',str(x)))
# msplit[curation_col].explode().value_counts()

In [77]:
msplit[500:1000]

external_id       Q:width
939   5637181594    ["8.5 in"]
940   5637181591    ["8.5 in"]
941   5637181589    ["8.5 in"]
942   5637181587    ["8.5 in"]
954   5637170031    ["1.5 in"]
958   5637440524   ["2.75 in"]
959   5637440522   ["3.25 in"]
960   5637440520   ["5.56 in"]
961   5637551601     ["12 in"]
965   5637485927      ["2 in"]
972   5637467853   ["4.75 in"]
973   5637181588    ["8.5 in"]
981   5637563650    ["8.5 in"]
982   5637563649    ["8.5 in"]
985   5637423252   ["6.97 in"]
1005  5637511796    ["1.5 in"]
1006  5637511795    ["1.5 in"]
1007  5637511794    ["1.5 in"]
1008  5637511787   ["0.38 in"]
1009  5637511771     ["20 in"]
1019  5637209929     ["43 in"]
1020  5637209928     ["40 in"]
1021  5637209927     ["40 in"]
1030  5637479520     ["25 in"]
1058  5637421948    ["8.5 in"]
1061  5637391033   ["8.62 in"]
1064  5637462248     ["15 in"]
1069  5637243977     ["11 in"]
1070  5637243976     ["11 in"]
1071  5637243975     ["11 in"]
1072  5637243972    ["8.5 in"]
1076  5637510758   ["0.62 in"]
1078  5637506768    ["8.5 in"]
1082  5637178762   ["4.12 in"]
1083  5637178760    ["9.5 in"]
1084  5637178751    ["8.5 in"]
1085  5637178740   ["9.25 in"]
1086  5637178739   ["9.25 in"]
1094  5637206457    ["8.5 in"]
1095  5637465215    ["8.5 in"]
1102  5637388628     ["38 in"]
1114  5637323153      ["6 in"]
1116  5637241469     ["13 in"]
1117  5637241395    ["8.5 in"]
1118  5637241393      ["4 in"]
1122  5637431475      ["9 in"]
1124  5637431472      ["9 in"]
1125  5637431471      ["9 in"]
1126  5637431466      ["9 in"]
1127  5637431465      ["9 in"]
1128  5637431464     ["15 in"]
1129  5637431463     ["12 in"]
1130  5637431461     ["11 in"]
1131  5637431460      ["9 in"]
1132  5637431457      ["5 in"]
1133  5637431456      ["4 in"]
1134  5637180160    ["8.5 in"]
1136  5637180156    ["8.5 in"]
1138  5637467474      ["5 in"]
1139  5637467473      ["5 in"]
1142  5637439342    ["8.5 in"]
1158  5637392706     ["16 in"]
1161  5637420174    ["8.5 in"]
1176  5637392713     ["18 in"]
1182  5637208893     ["12 in"]
1183  5637208646    ["8.5 in"]
1193  5637466322   ["4.19 in"]
1194  5637466022  ["10.57 in"]
1211  5637519086    ["8.5 in"]
1212  5637519085    ["8.5 in"]
1231  5637521743    ["8.5 in"]
1232  5637561348     ["11 in"]
1238  5637546173    ["1.5 in"]
1251  5637510242   ["0.38 in"]
1253  5637423632     ["33 in"]
1267  5637152161   ["3.25 in"]
1270  5637529143     ["14 in"]
1271  5637529142     ["14 in"]
1272  5637529141      ["7 in"]
1273  5637529138      ["7 in"]
1279  5637369335      ["9 in"]
1280  5637369296  ["12.62 in"]
1281  5637209924     ["33 in"]
1282  5637209921     ["40 in"]
1283  5637209917     ["33 in"]
1284  5637209913     ["11 in"]
1285  5637209893      ["6 in"]
1301  5637347713   ["9.38 in"]
1302  5637529113      ["7 in"]
1303  5637529112      ["7 in"]
1304  5637529105     ["12 in"]
1305  5637529104      ["7 in"]
1306  5637529102     ["12 in"]
1307  5637388890     ["16 in"]
1308  5637388779     ["24 in"]
1309  5637388771     ["30 in"]
1310  5637388751      ["7 in"]
1311  5637422653    ["8.5 in"]
1312  5637422645    ["8.5 in"]
1313  5637422644    ["8.5 in"]
1314  5637422634    ["8.5 in"]
1321  5637235011     ["11 in"]
1323  5637348272   ["9.25 in"]
1324  5637348270   ["9.25 in"]
1327  5637348262   ["6.53 in"]
1336  5637387374   ["5.88 in"]
1340  5637489050      ["8 in"]
1372  5637503555     ["19 in"]
1373  5637175742   ["5.56 in"]
1374  5637175733   ["8.38 in"]
1375  5637175731   ["5.56 in"]
1376  5637175706    ["8.5 in"]
1377  5637175705    ["5.5 in"]
1378  5637175704    ["8.5 in"]
1379  5637175701    ["8.5 in"]
1380  5637175700    ["8.5 in"]
1393  5637417969     ["11 in"]
1394  5637417968     ["11 in"]
1395  5637417966    ["8.5 in"]
1396  5637417965    ["8.5 in"]
1397  5637417961    ["8.5 in"]
1398  5637417960     ["11 in"]
1399  5637417959     ["11 in"]
1400  5637417958    ["8.5 in"]
1401  5637417957    ["8.5 in"]
1457  5637232735     ["30 in"]
1458  5637232673    ["8.5 in"]
1461  5637383086    ["8.5 in"]
1478  

# send to the folder for upload

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

In [ ]:
looks_good('WBMason', attribute, df, matcheswidthcallout)